In [5]:
import numpy as np
from pyblock2._pyscf.ao2mo import integrals as itg
from pyblock2.driver.core import DMRGDriver, SymmetryTypes

In [29]:
from pyscf import gto, scf, fci, cc

a = 2 #0.96892 # 1.831aB
nH = 4
atoms = ""
for n in range(nH):
    atoms += f"H {n*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="ccpvdz", symmetry="c1", unit="bohr")
mf = scf.RHF(mol).density_fit()
mf.kernel()

## non-df e_fci = -5.42273347731
## df e_fci = -5.4276602955

# mycc = cc.CCSD(mf)
# mycc.kernel()
# e_pt = mycc.ccsd_t()
# print(mycc.e_tot+e_pt)

converged SCF energy = -2.15321137072455


-2.153211370724548

In [30]:
# print(mf._eri)
from pyscf import df, lib
chol_df = df.incore.cholesky_eri(mol, mf.with_df.auxmol.basis)
chol_df = lib.unpack_tril(chol_df).reshape(chol_df.shape[0], -1)
chol_df = chol_df.reshape((-1, mol.nao, mol.nao))
eri_ao_df = np.einsum('lpq,lrs->pqrs', chol_df, chol_df)
mf._eri = eri_ao_df
# print(mf._eri.shape)

In [31]:
from pyscf import lo
mf.mo_coeff = lo.orth.lowdin(mf.mol.intor('cint1e_ovlp_sph'))
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

In [33]:
mycc = cc.CCSD(mf)
mycc.kernel()
e_pt = mycc.ccsd_t()
print(mycc.e_tot+e_pt)


WARN:  diis singular, eigh(h) [-6.23696897e-03 -1.35295722e-08 -4.82000726e-10  5.66777730e-12
  3.76517895e-10  8.01845013e+02  5.46148040e+08]



LinAlgError: Singular matrix

In [22]:
bond_dims = [20] * 2 + [40] * 2 + [60] * 2 + [80]*2 +[100]*4
noises = [1e-4] * 4 + [1e-5] * 4 + [0]
thrds = [1e-6] * 8 + [1e-7]

In [26]:
occs = np.diag(mycc.make_rdm1())

In [34]:
-11.1871 - -10.6973805689852

-0.48971943101479987

In [27]:
bias = 0.2
print(occs)
occs = occs + bias * (occs < 1) - bias * (occs > 1)
print(occs)

[1.95407201e+00 1.90816419e+00 7.49156216e-02 1.36604866e-02
 6.48067456e-03 1.11590068e-02 7.69567005e-03 1.09919272e-02
 2.31619099e-03 2.31619099e-03 1.76793068e-03 1.76793068e-03
 1.96545524e-03 3.97153170e-04 3.97153170e-04 9.73034282e-04
 2.13712229e-04 2.13712229e-04 4.34291159e-04 9.76569999e-05]
[1.75407201 1.70816419 0.27491562 0.21366049 0.20648067 0.21115901
 0.20769567 0.21099193 0.20231619 0.20231619 0.20176793 0.20176793
 0.20196546 0.20039715 0.20039715 0.20097303 0.20021371 0.20021371
 0.20043429 0.20009766]


In [ ]:
bias = 0.3 # make it more random
# occs = np.diag(mycc.make_rdm1())
# occs = occs + bias * (occs < 1) - bias * (occs > 1)
occs = np.array([2]*int(nH/2)+[0]*(mol.nao-nH))
# occs = occs + bias * (occs < 1) - bias * (occs > 1)
# print(occs)

[1.7 1.7 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3]


In [28]:
# bias = 0.3 # make it more random
# # occs = np.diag(mycc.make_rdm1())
# # occs = occs + bias * (occs < 1) - bias * (occs > 1)
# occs = np.array([2]*int(nH/2)+[0]*(mol.nao-nH))
# # occs = occs + bias * (occs < 1) - bias * (occs > 1)

ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,
    ncore=0, ncas=None, g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=0)
ket = driver.get_random_mps(tag="GS", bond_dim=250, occs=occs, nroots=1)
energy = driver.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)
print('DMRG energy = %20.15f' % energy)


Sweep =    0 | Direction =  forward | Bond dimension =   20 | Noise =  1.00e-04 | Dav threshold =  1.00e-06
Time elapsed =      2.396 | E =      -2.2419225582 | DW = 1.70567e-05

Sweep =    1 | Direction = backward | Bond dimension =   20 | Noise =  1.00e-04 | Dav threshold =  1.00e-06
Time elapsed =      4.543 | E =      -2.2424605952 | DE = -5.38e-04 | DW = 3.55885e-05

Sweep =    2 | Direction =  forward | Bond dimension =   40 | Noise =  1.00e-04 | Dav threshold =  1.00e-06
Time elapsed =      7.254 | E =      -2.2430027727 | DE = -5.42e-04 | DW = 3.60333e-07

Sweep =    3 | Direction = backward | Bond dimension =   40 | Noise =  1.00e-04 | Dav threshold =  1.00e-06
Time elapsed =     10.111 | E =      -2.2430233940 | DE = -2.06e-05 | DW = 6.08356e-07

Sweep =    4 | Direction =  forward | Bond dimension =   60 | Noise =  1.00e-05 | Dav threshold =  1.00e-06
Time elapsed =     13.494 | E =      -2.2430433064 | DE = -1.99e-05 | DW = 1.92424e-08

Sweep =    5 | Direction = backward 

In [14]:
ncas, n_elec, spin, ecore, h1e, g2e, orb_sym = itg.get_rhf_integrals(mf,g2e_symm=8)

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SU2, n_threads=4)
driver.initialize_system(n_sites=ncas, n_elec=n_elec, spin=spin, orb_sym=orb_sym)

mpo = driver.get_qc_mpo(h1e=h1e, g2e=g2e, ecore=ecore, iprint=1)
ket = driver.get_random_mps(tag="GS", bond_dim=250, nroots=1,occs=[1]*nH)
energy = driver.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises,
    thrds=thrds, iprint=1)
print('DMRG energy = %20.15f' % energy)

# pdm1 = driver.get_1pdm(ket)
# pdm2 = driver.get_2pdm(ket).transpose(0, 3, 1, 2)
# print('Energy from pdms = %20.15f' % (np.einsum('ij,ij->', pdm1, h1e)
#     + 0.5 * np.einsum('ijkl,ijkl->', pdm2, driver.unpack_g2e(g2e)) + ecore))

# impo = driver.get_identity_mpo()
# expt = driver.expectation(ket, mpo, ket) / driver.expectation(ket, impo, ket)
# print('Energy from expectation = %20.15f' % expt)

integral symmetrize error =  0.0
integral cutoff error =  8.638844320010619e-19
mpo terms =      88246

Build MPO | Nsites =    20 | Nterms =      88246 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    20 .. Mmpo =    14 DW = 0.00e+00 NNZ =       14 SPT = 0.0000 Tmvc = 0.004 T = 0.013
 Site =     1 /    20 .. Mmpo =    50 DW = 0.00e+00 NNZ =       56 SPT = 0.9200 Tmvc = 0.006 T = 0.023
 Site =     2 /    20 .. Mmpo =    84 DW = 0.00e+00 NNZ =      998 SPT = 0.7624 Tmvc = 0.006 T = 0.023
 Site =     3 /    20 .. Mmpo =   114 DW = 0.00e+00 NNZ =     1247 SPT = 0.8698 Tmvc = 0.007 T = 0.026
 Site =     4 /    20 .. Mmpo =   152 DW = 0.00e+00 NNZ =     1932 SPT = 0.8885 Tmvc = 0.006 T = 0.031
 Site =     5 /    20 .. Mmpo =   198 DW = 0.00e+00 NNZ =     2706 SPT = 0.9101 Tmvc = 0.008 T = 0.035
 Site =     6 /    20 .. Mmpo =   252 DW = 0.00e+00 NNZ =     3534 SPT = 0.9292 Tmvc = 0.009 T = 0.034
 Site =     7 /    20 .. Mmpo =   314 DW = 0.00e+00 NNZ =     4379 SPT = 0.9447 Tmvc

In [ ]:
# non df
# DMRG energy =   -5.422733477309043
# Energy from pdms =   -5.422733477309011
# Energy from expectation =   -5.422733477309038

In [ ]:
# df
# DMRG energy =   -5.427660295498061
# Energy from pdms =   -5.427660295498059
# Energy from expectation =   -5.427660295498058